### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [ ]:

import io
import avro.schema
import avro.io
import avro.datafile
import avro.errors
schema_str="""
{
"type":"record",
"name":"User",
"fields":[
{"name":"id","type":"long"},
{"name":"name","type":"string"},
{"name":"email","type":["null","string"],"default":null},
{"name":"address", "type":{
"type":"record",
"name":"Address",
"fields":[
{"name":"street","type":"string"},
{"name":"city","type":"string"},
{"name":"zip","type":"string"}
]
}},
{"name":"phone_numbers","type":{"type":"array","items":"string"}}
]
}
"""
parsed_schema=avro.schema.parse(schema_str)
valid_data={
"id":12345,
"name":"John Doe",
"email":"john.doe@example.com",
"address":{
"street":"123 Main St",
"city":"Anytown",
"zip":"12345"
},
"phone_numbers":["123-456-7890","098-765-4321"]
}
invalid_data_missing_field={
"id":67890,
"name":"Jane Doe",
"address":{
"street":"456 Oak Ave",
"city":"Otherville",
"zip":"67890"
},
"phone_numbers":["987-654-3210"]
}
invalid_data_wrong_type={
"id":54321,
"name":"Peter Pan",
"email":12345,
"address":{
"street":"789 Pine Ln",
"city":"Neverland",
"zip":"34567"
},
"phone_numbers":["111-222-3333"]
}
valid_data_no_email={
"id":22222,
"name":"Test User",
"email":None,
"address":{
"street":"555 Sample Rd",
"city":"Sampleton",
"zip":"54321"
},
"phone_numbers":[]
}
file_writer=io.BytesIO()
datum_writer=avro.io.DatumWriter(parsed_schema)
writer=avro.datafile.DataFileWriter(file_writer,datum_writer,parsed_schema)
print("Attemptingtowritevaliddata:")
try:
    writer.append(valid_data)
    print("Validdatawrittensuccessfully.")
except avro.errors.AvroTypeException as e:
    print(f"Errorwritingvaliddata:{e}")
writer.flush()
print("\nAttemptingtowritevaliddata(noemail):")
try:
    writer.append(valid_data_no_email)
    print("Validdata(noemail)writtensuccessfully.")
except avro.errors.AvroTypeException as e:
    print(f"Errorwritingvaliddata(noemail):{e}")
writer.flush()
print("\nAttemptingtowriteinvaliddata(missingfield):")
try:
    writer.append(invalid_data_missing_field)
    print("Invaliddata(missingfield)writtensuccessfully(shouldfail).")
except avro.errors.AvroTypeException as e:
    print(f"Errorwritinginvaliddata(missingfield):{e}")
writer.flush()
print("\nAttemptingtowriteinvaliddata(wrongtype):")
try:
    writer.append(invalid_data_wrong_type)
    print("Invaliddata(wrongtype)writtensuccessfully(shouldfail).")
except avro.errors.AvroTypeException as e:
    print(f"Errorwritinginvaliddata(wrongtype):{e}")
writer.flush()
avro_data=file_writer.getvalue()
writer.close()
print("\nReadingbackdatafromBytesIO:")
file_reader=io.BytesIO(avro_data)
reader=avro.datafile.DataFileReader(file_reader,avro.io.DatumReader())
read_records=[]
try:
    for record in reader:
        read_records.append(record)
    print("Successfullyreadrecords:")
    for rec in read_records:
        print(rec)
except Exception as e:
    print(f"Errorreadingdata:{e}")
reader.close()

Attemptingtowritevaliddata:
Validdatawrittensuccessfully.

Attemptingtowritevaliddata(noemail):
Validdata(noemail)writtensuccessfully.

Attemptingtowriteinvaliddata(missingfield):
Invaliddata(missingfield)writtensuccessfully(shouldfail).

Attemptingtowriteinvaliddata(wrongtype):
Errorwritinginvaliddata(wrongtype):The datum "12345" provided for "email" is not an example of the schema [
  "null",
  "string"
]

ReadingbackdatafromBytesIO:
Successfullyreadrecords:
{'id': 12345, 'name': 'John Doe', 'email': 'john.doe@example.com', 'address': {'street': '123 Main St', 'city': 'Anytown', 'zip': '12345'}, 'phone_numbers': ['123-456-7890', '098-765-4321']}
{'id': 22222, 'name': 'Test User', 'email': None, 'address': {'street': '555 Sample Rd', 'city': 'Sampleton', 'zip': '54321'}, 'phone_numbers': []}
{'id': 67890, 'name': 'Jane Doe', 'email': None, 'address': {'street': '456 Oak Ave', 'city': 'Otherville', 'zip': '67890'}, 'phone_numbers': ['987-654-3210']}
